## Objectif du notebook :

Créer des fonctions qui oeuvreront au même moment que la NER pour protéger des instances sensibles qui ne sont pas des entités nommées, comme !
- Des numéros de sécurité sociale
- Des sites internet
- Des adresses mail
- Des plaques d'immatriculation

In [1]:
# Imports

import regex as re
import os
import pandas as pd

In [2]:
# Création de la liste de groupes d'entités :

entity_groups = ["SS", "email", "plate", "phone", "date"]

In [3]:
# Regex pour les plaques :

reg_plate = r"[A-Z]+[A-Z0-9\- ]{6,8}"

# Regex pour les numéros

reg_num = r"[0-9+/.\- ]{6,}"

# Regex pour un email :

reg_email = r"[a-zA-Z0-9.\-\_]+@[a-zA-Z0-9.\-\_]+"

In [4]:
# Fonction qui ajoute les entités non nommées à la liste de NER

def add_entities (text:str,
                  l : list = None):
    if not l:
        l = []
    plates = re.findall(reg_plate, text)
    plates = list(dict.fromkeys(plates))
    numbers = re.findall(reg_num, text)
    numbers = list(dict.fromkeys(numbers))
    emails = re.findall(reg_email, text)
    emails = list(dict.fromkeys(emails))
    entity_groups = {"Number":numbers,"email":emails,"plate":plates}
    
    for entity_type in entity_groups.keys():
        for item in entity_groups[entity_type]:
            if isinstance(item, tuple) and tuple[0] != '':
                dic = {}
                dic["entity_group"] = entity_type
                dic["word"] = item[0]
                l.append(dic)
            elif isinstance(item, str):
                dic = {}
                dic["entity_group"] = entity_type
                dic["word"] = item
                l.append(dic)
    return(l)

In [5]:
# Fonctions pour la reconnaissance d'entités nommées

from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")

from transformers import pipeline

nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

def identify_ners(text:str) -> list :
    ner = nlp(text)
    ner = add_entities(text, ner)
    return(ner)
    
# Fonctions pour la pseudonymisation

def pseudo(name:str):
    return('X'*len(name))

def pseudo_all(text : str , file_ners: list):
    """
    Pseudonymizes all NERs within the file."""
    for i, ner in enumerate(file_ners):
        if ner["entity_group"]:
            name = ner["word"]
            if len(name) > 1:
                pseudo_name = pseudo(name)
                text = text.replace(name, pseudo_name)
    return(text)

/opt/anaconda3/envs/pdf_pseudo_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Test

text = "j'habite au 2 route tru truc à machin"

ners = identify_ners(text)

text_new = pseudo_all(text, ners)

text_new

"j'habite au 2 route XXXXXXXX à XXXXXX"

Si utilisé sur onyxia :

In [7]:
"""
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "projet-transformers"
FILE_KEY_S3 = "Nom fichier.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    df_dila = pd.read_csv(file_in, 
            sep=";",
            encoding='utf-8',
            usecols = ["N°","Message d'origine"],
            index_col=0
            )"""

'\n# Create filesystem object\nS3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]\nfs = s3fs.S3FileSystem(client_kwargs={\'endpoint_url\': S3_ENDPOINT_URL})\nBUCKET = "projet-transformers"\nFILE_KEY_S3 = "01092023 - Tests Anonymisation - Outil DINUM 2.csv"\nFILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3\n\nwith fs.open(FILE_PATH_S3, mode="rb") as file_in:\n    df_dila = pd.read_csv(file_in, \n            sep=";",\n            encoding=\'utf-8\',\n            usecols = ["N°","Message d\'origine"],\n            index_col=0\n            )'

Sinon

In [8]:
file_in = "../data/test.csv"

In [9]:
df_dila = pd.read_csv(file_in, 
            sep=";",
            encoding='utf-8',
            usecols = ["N°","Message d'origine"],
            index_col=0
            )

df_dila.head()

,Message d'origine
N°,
1,Je suis madame Truc et j'ai une question sur …


In [10]:
# Fonctions pour process tout le texte

def process_text(sentence :str):
    ners = identify_ners(sentence)
    new_sentence = pseudo_all(sentence, ners)
    return(new_sentence)
    

df_dila["Message d'origine"] = df_dila["Message d'origine"].apply(process_text)

In [11]:
"""BUCKET_OUT = "projet-transformers"
FILE_KEY_OUT_S3 = "test_dila_output.csv"
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    df_dila.to_csv(file_out,
    #encoding="utf-8",
    #sep=";"
    )"""


# Sinon

file_out = "../data/output.csv"
df_dila.to_csv(file_out)

In [12]:
df_dila

,Message d'origine
N°,
1,Je suis madame XXXX et j'ai une question sur …
